In [23]:
import pandas as pd
from io import StringIO
import requests
from tqdm import tqdm
import ast

df=pd.read_csv(r"datasets/01a_Sentence_df.csv", sep="|", lineterminator='\n')
df["taggedSentences"]  = [ast.literal_eval(text) for text in df["taggedSentences"]]
display(df.head())

df['Level_ABC'] = df.apply (lambda row: (row["newLevel"]),axis=1)
df = df[df.Level_ABC != "L"]
level = df.groupby('Level_ABC')
print(level.size())
df = level.apply(lambda x: x.sample(level.size().min()).reset_index(drop=True))
print(len(df[df["newLevel"]=="A"]))

,Text,newLevel,taggedSentences,parsedSentences
0,Zu meiner Familie gehören vier Personen.,A,"[[Zu, APPR, Dat], [meiner, PRO, Poss, Attr, -,...",(ROOT (S (PP (APPR Zu) (PPOSAT meiner) (NN Fam...
1,Die Mutter bin ich und dann gehört natürlich m...,A,"[[Die, ART, Def, Nom, Sg, Fem], [Mutter, N, Re...",(ROOT (CS (S (NP (ART Die) (NN Mutter)) (VAFIN...
2,"Wir haben zwei Kinder, einen Sohn, der sechs J...",A,"[[Wir, PRO, Pers, Subst, 1, Nom, Pl, *], [habe...",(ROOT (S (PPER Wir) (VAFIN haben) (CNP (NP (CA...
3,Wir wohnen in einem kleinen Haus mit einem Gar...,A,"[[Wir, PRO, Pers, Subst, 1, Nom, Pl, *], [wohn...",(ROOT (S (PPER Wir) (VVFIN wohnen) (PP (APPR i...
4,Dort können die Kinder ein bisschen spielen.,A,"[[Dort, ADV], [können, VFIN, Mod, 3, Pl, Pres,...",(ROOT (S (ADV Dort) (VMFIN können) (NP (ART di...


Level_ABC
A     7141
B    20087
C    19318
dtype: int64
7141


In [24]:
from sklearn.model_selection import train_test_split

def save_df(path,df):
    df.to_csv(path,sep="|",index=False)
    

x_train ,x_test = train_test_split(df,test_size=0.3)

for df_inProcessing, df_name in tqdm([(x_train, "train"), (x_test, "test")]):
    for N in tqdm(range(1,10)): # number of sentences per Text
        ###Text with len N sentence###
        rows = []

        ###split training and test data###


        complexityLevels = ['A', 'B', 'C']
        
        no_of_text = min([int(len(df_inProcessing[(df["newLevel"] == complexityLevel)])/N) for complexityLevel in complexityLevels])
        print(no_of_text)
        for complexityLevel in complexityLevels:
            
            for n in tqdm(range(no_of_text)): #number of text per level
                df_sample = df_inProcessing[(df["newLevel"] == complexityLevel)].sample(N)
                rows.append([" ".join(df_sample['Text'].tolist()), complexityLevel, df_sample['taggedSentences'].tolist(), df_sample['parsedSentences'].tolist()])
        random_df = pd.DataFrame(rows, columns=['Text', 'Level', 'RFTagger', 'parsedText'])

        display(random_df.head())

        filename = str("datasets/RandomText/10_RandomText_" + df_name + "_" + str(N) + "_df.csv")
        save_df(filename, random_df)













  0%|          | 0/2 [00:00<?, ?it/s]












  0%|          | 0/9 [00:00<?, ?it/s]/home/soeren/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.














  0%|          | 0/4955 [00:00<?, ?it/s]/home/soeren/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.














  0%|          | 12/4955 [00:00<00:41, 117.77it/s]

4955
















  1%|          | 25/4955 [00:00<00:40, 122.22it/s]













  1%|          | 37/4955 [00:00<00:41, 118.61it/s]













  1%|          | 48/4955 [00:00<00:42, 116.13it/s]













  1%|▏         | 63/4955 [00:00<00:40, 121.92it/s]













  2%|▏         | 75/4955 [00:00<00:40, 121.42it/s]













  2%|▏         | 90/4955 [00:00<00:39, 124.55it/s]













  2%|▏         | 102/4955 [00:00<00:39, 123.66it/s]













  2%|▏         | 116/4955 [00:00<00:38, 124.57it/s]













  3%|▎         | 131/4955 [00:01<00:38, 126.48it/s]













  3%|▎         | 144/4955 [00:01<00:38, 125.24it/s]













  3%|▎         | 157/4955 [00:01<00:38, 124.04it/s]













  3%|▎         | 170/4955 [00:01<00:38, 124.24it/s]













  4%|▎         | 183/4955 [00:01<00:38, 123.44it/s]













  4%|▍         | 198/4955 [00:01<00:38, 125.04it/s]













  4%|▍         | 213/4955 [00:01<00:37, 126.13it/s]













  5%|▍         | 

 24%|██▍       | 1193/4955 [00:08<00:26, 144.37it/s]













 24%|██▍       | 1210/4955 [00:08<00:25, 144.63it/s]













 25%|██▍       | 1227/4955 [00:08<00:25, 144.87it/s]













 25%|██▌       | 1243/4955 [00:08<00:25, 144.74it/s]













 25%|██▌       | 1260/4955 [00:08<00:25, 144.99it/s]













 26%|██▌       | 1277/4955 [00:08<00:25, 145.24it/s]













 26%|██▌       | 1294/4955 [00:08<00:25, 145.45it/s]













 26%|██▋       | 1311/4955 [00:09<00:25, 145.57it/s]













 27%|██▋       | 1327/4955 [00:09<00:24, 145.67it/s]













 27%|██▋       | 1344/4955 [00:09<00:24, 145.89it/s]













 27%|██▋       | 1360/4955 [00:09<00:24, 146.01it/s]













 28%|██▊       | 1376/4955 [00:09<00:24, 146.12it/s]













 28%|██▊       | 1393/4955 [00:09<00:24, 146.31it/s]













 28%|██▊       | 1410/4955 [00:09<00:24, 146.52it/s]













 29%|██▉       | 1427/4955 [00:09<00:24, 146.65it/s]













 29%|██▉  

 48%|████▊     | 2402/4955 [00:16<00:17, 148.86it/s]













 49%|████▉     | 2418/4955 [00:16<00:17, 148.78it/s]













 49%|████▉     | 2434/4955 [00:16<00:16, 148.77it/s]













 49%|████▉     | 2450/4955 [00:16<00:16, 148.82it/s]













 50%|████▉     | 2466/4955 [00:16<00:16, 148.88it/s]













 50%|█████     | 2483/4955 [00:16<00:16, 148.99it/s]













 50%|█████     | 2499/4955 [00:16<00:16, 149.03it/s]













 51%|█████     | 2515/4955 [00:16<00:16, 149.07it/s]













 51%|█████     | 2531/4955 [00:16<00:16, 149.12it/s]













 51%|█████▏    | 2547/4955 [00:17<00:16, 149.15it/s]













 52%|█████▏    | 2563/4955 [00:17<00:16, 149.18it/s]













 52%|█████▏    | 2579/4955 [00:17<00:15, 149.24it/s]













 52%|█████▏    | 2595/4955 [00:17<00:15, 149.26it/s]













 53%|█████▎    | 2611/4955 [00:17<00:15, 149.27it/s]













 53%|█████▎    | 2627/4955 [00:17<00:15, 149.08it/s]













 53%|█████

...














 89%|████████▉ | 8/9 [01:35<00:11, 11.88s/it]













  0%|          | 0/231 [00:00<?, ?it/s]













  8%|▊         | 19/231 [00:00<00:01, 182.49it/s]

231
















 16%|█▌        | 36/231 [00:00<00:01, 175.37it/s]













 21%|██        | 48/231 [00:00<00:01, 153.07it/s]













 26%|██▌       | 59/231 [00:00<00:01, 143.89it/s]













 34%|███▍      | 78/231 [00:00<00:01, 152.29it/s]













 42%|████▏     | 97/231 [00:00<00:00, 157.56it/s]













 50%|█████     | 116/231 [00:00<00:00, 161.97it/s]













 58%|█████▊    | 135/231 [00:00<00:00, 165.05it/s]













 66%|██████▌   | 152/231 [00:00<00:00, 161.97it/s]













 74%|███████▍  | 171/231 [00:01<00:00, 164.14it/s]













 82%|████████▏ | 190/231 [00:01<00:00, 166.23it/s]













 91%|█████████ | 210/231 [00:01<00:00, 168.63it/s]













100%|█████████▉| 230/231 [00:01<00:00, 170.65it/s]













100%|██████████| 231/231 [00:01<00:00, 170.45it/s]













  0%|          | 0/231 [00:00<?, ?it/s]













  7%|▋         | 17/231 [00:00<00:01, 161.19it/s]













 16%|█▌        | 36/231 [00:00<00:01, 171.7

,Text,Level,RFTagger,parsedText
0,Ich habe wenig Geld. Ich habe ein Handy! Sag m...,A,"[[[Ich, PRO, Pers, Subst, 1, Nom, Sg, *], [hab...",[(ROOT (S (PPER Ich) (VAFIN habe) (NP (PIAT we...
1,Ich habe in Lancieren seit eine kurze Zeit gew...,A,"[[[Ich, PRO, Pers, Subst, 1, Nom, Sg, *], [hab...",[(ROOT (CS (S (PPER Ich) (VAFIN habe) (PP (APP...
2,Viel Glück! Er sprecht die Sprache nicht so gu...,A,"[[[Viel, PRO, Indef, Attr, -, *, *, *], [Glück...",[(ROOT (NP (AP (PIDAT Viel)) (NN Glück) ($. !)...
3,Wenn ich den Radio höre oder die Zeitung lesen...,A,"[[[Wenn, CONJ, SubFin, -], [ich, PRO, Pers, Su...",[(ROOT (CS (S (KOUS Wenn) (PPER ich) (NP (ART ...
4,"Wie gehst dir, Ich wünsche das seine Frau hat ...",A,"[[[Wie, PROADV, Inter], [gehst, VFIN, Full, 2,...",[(ROOT (S (PWAV Wie) (VVFIN gehst) (PPER dir) ...















100%|██████████| 9/9 [01:39<00:00, 11.07s/it]
























100%|██████████| 2/2 [06:44<00:00, 202.03s/it]













In [ ]:
int(len(df_inProcessing[(df["newLevel"] == complexityLevel)])/5))